In [ ]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import fsspec

warnings.simplefilter('ignore') # filter some warning messages
xr.set_options(display_style="html")  #display dataset nicely

In [ ]:
file_opendap = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2015/001/20150101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
file_local = './../../data/20150101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
file_aws = 'https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1'
file_aws2 = 'https://mur-sst.s3.us-west-2.amazonaws.com/zarr'

In [ ]:
%%time

ds_aws = xr.open_zarr(file_aws,consolidated=True)
ds_aws = ds_aws.sel(time='2015-01-01')
ds_aws

In [ ]:
%%time

ds_aws2 = xr.open_zarr(file_aws2,consolidated=True)
ds_aws2 = ds_aws.sel(time='2015-01-01')
ds_aws2

In [ ]:
ds_odap = xr.open_dataset(file_opendap)
ds_odap

In [ ]:
ds_local = xr.open_dataset(file_local)
ds_local

In [ ]:
ds_local.analysed_sst.sel(lat=slice(20,30),lon=slice(-120,-100)).isel(time=0).plot()

In [ ]:
# print out value near land
xlat = 24
xlon = -100
print(ds_local.analysed_sst.sel(lat=24,lon=-100).isel(time=0).data)
print(ds_odap.analysed_sst.sel(lat=24,lon=-100).isel(time=0).data)
print(ds_aws.analysed_sst.sel(lat=24,lon=-100).isel(time=0).compute().data)
print(ds_aws2.analysed_sst.sel(lat=24,lon=-100).isel(time=0).compute().data)

In [ ]:
ds_local.analysed_sst.sel(lat=slice(20,30),lon=slice(100,120)).isel(time=0).plot()

In [ ]:
# print out value near land
xlat = 21
xlon = 101
print(ds_local.analysed_sst.sel(lat=24,lon=-100).isel(time=0).data)
print(ds_odap.analysed_sst.sel(lat=24,lon=-100).isel(time=0).data)
print(ds_aws.analysed_sst.sel(lat=24,lon=-100).isel(time=0).compute().data)
print(ds_aws2.analysed_sst.sel(lat=24,lon=-100).isel(time=0).compute().data)